In [1]:
import pandas as pd
import numpy as np

In [2]:
BRL_indices = pd.read_excel('/data2/project_BRL/project_tDCS/mri_session_checklist_data.xlsx')

In [97]:
BRL_cov = BRL_indices.iloc[:,:3] 

In [4]:
BRL_cov.columns = ['Session Name','male','age']

In [16]:
sublist = []
for i in range(len(BRL_cov)):
    if i > 0:
        tmp = BRL_cov['Session Name'][i]
        tmp_ = tmp.split('_1')[0].split('BRL')[-1]
        sub = np.int(tmp_)
        sublist.append(sub)

In [25]:
BRL_cov['sub'] = [0]+sublist

/home/kimhj9502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
BRL_cov = BRL_cov.replace('M',1).replace('F',0)

In [29]:
BRL_cov.columns

Index(['Session Name', 'male', 'age', 'sub'], dtype='object')

In [10]:
pwd

'/home/kimhj9502/Project/project_tDCS/analysis'

# Modeling

In [12]:
model_a = pd.read_csv('/home/kimhj9502/Project/project_tDCS/analysis/modeling/modeling_anode_tmp.csv')

In [13]:
model_s = pd.read_csv('/home/kimhj9502/Project/project_tDCS/analysis/modeling/modeling_sham_tmp.csv')

In [32]:
model_a['sub'] = model_a['subj_list_anode'] 
model_s['sub'] = model_s['subj_list_sham'] 

In [36]:
model_ss = pd.merge(BRL_cov, model_s, on = 'sub')
model_aa = pd.merge(BRL_cov, model_a, on = 'sub')

In [41]:
model = pd.concat([model_ss, model_aa])

/home/kimhj9502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [42]:
from statsmodels.genmod.generalized_linear_model import GLM

In [44]:
model.columns

Index(['Session Name', 'Unnamed: 0', 'age', 'b', 'ep', 'male', 'piPun',
       'piRew', 'rhoPun', 'rhoRew', 'sub', 'subj_list_anode', 'subj_list_sham',
       'xi'],
      dtype='object')

In [59]:
anode = np.isnan(model['subj_list_anode']).tolist()

In [71]:
model['anode_'] = anode

In [72]:
model['anode'] = model['anode_'] * 1

In [74]:
X = model[['anode','age', 'male']]

In [76]:
lny = model['piPun']

In [79]:
glm = GLM(lny, X)

In [81]:
rs = glm.fit()

In [84]:
summ = rs.summary()

In [88]:
summ.as_csv

<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
Dep. Variable:                  piPun   No. Observations:                   31
Model:                            GLM   Df Residuals:                       28
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                         0.23626
Method:                          IRLS   Log-Likelihood:                -20.046
Date:                Tue, 13 Jul 2021   Deviance:                       6.6153
Time:                        15:17:11   Pearson chi2:                     6.62
No. Iterations:                     3   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
anode          0.4276      0.181      2.357      0.

In [93]:
for y_ in ['xi', 'b', 'ep', 'piRew', 'piPun','rhoPun', 'rhoRew']:
    lny = model[y_]
    glm = GLM(lny, X)
    rs = glm.fit()
    summ = rs.summary()
    print(y_)
    print(summ.as_csv)

xi
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
Dep. Variable:                     xi   No. Observations:                   31
Model:                            GLM   Df Residuals:                       28
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                      0.00040996
Method:                          IRLS   Log-Likelihood:                 78.482
Date:                Tue, 13 Jul 2021   Deviance:                     0.011479
Time:                        15:20:36   Pearson chi2:                   0.0115
No. Iterations:                     3   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
anode          0.0577      0.008      7.630     

In [ ]:
# xi, b, piPun, and rhoPun 까지 차이가 있는 것으로 나옴

# Pav Bias

In [98]:
behav_1 = pd.read_csv('/home/kimhj9502/Project/project_tDCS/analysis/behav/data_subj_all_tmp1.csv')
behav_2 = pd.read_csv('/home/kimhj9502/Project/project_tDCS/analysis/behav/data_subj_all_tmp2.csv')

In [103]:
behav_2.columns

Index(['subject', 'db', 'pavBias_rew', 'pavBias_pun'], dtype='object')

In [114]:
behav_2_a = behav_2[behav_2['db'] == 'anode']
behav_2_s = behav_2[behav_2['db'] == 'sham']
behav_1_a = behav_1[behav_1['db'] == 'anode']
behav_1_s = behav_1[behav_1['db'] == 'sham']

In [117]:
behav_2_a.sort_values(by = 'subject',  inplace= True)
behav_2_s.sort_values(by = 'subject',  inplace= True)
behav_1_a.sort_values(by = 'subject',  inplace= True)
behav_1_s.sort_values(by = 'subject',  inplace= True)

/home/kimhj9502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kimhj9502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/kimhj9502/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing 

In [124]:
behav_a = pd.merge(behav_1_a, behav_2_a, on = 'subject')
behav_s = pd.merge(behav_1_s, behav_2_s, on = 'subject')

In [128]:
behav_ = pd.concat([behav_s, behav_a])

In [129]:
behav_['sub'] = behav_['subject'] 

In [130]:
behav = pd.merge(BRL_cov, behav_, on = 'sub')

In [133]:
behav = behav.replace('sham',0).replace('anode',1)

In [134]:
behav['anode'] = behav['db_x']

In [136]:
X = behav[['anode','age', 'male']]

In [137]:
behav.columns

Index(['Session Name', 'male', 'age', 'sub', 'subject', 'db_x', 'pavBias',
       'db_y', 'pavBias_rew', 'pavBias_pun', 'anode'],
      dtype='object')

In [139]:
for y_ in ['pavBias','pavBias_rew', 'pavBias_pun']:
    lny = behav[y_]
    glm = GLM(lny, X)
    rs = glm.fit()
    summ = rs.summary()
    print(y_)
    print(summ.as_csv)

pavBias
<bound method Summary.as_csv of <class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
Dep. Variable:                pavBias   No. Observations:                   32
Model:                            GLM   Df Residuals:                       29
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                         0.17743
Method:                          IRLS   Log-Likelihood:                -16.164
Date:                Tue, 13 Jul 2021   Deviance:                       5.1453
Time:                        15:39:08   Pearson chi2:                     5.15
No. Iterations:                     3   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
anode         -0.2405      0.147     -1.639